# Encontrando insights de la UEFA Champions League

In [0]:
# importaciones
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
from functools import reduce

La Liga de Campeones de la UEFA, a menudo llamada Liga de Campeones, es una competencia anual de fútbol por excelencia que cautiva a los fanáticos de todo el mundo. Establecida en 1955 como la Copa de Clubes Campeones de Europa, evolucionó a la Liga de Campeones de la UEFA en 1992, ampliando su atractivo. El formato moderno cuenta con 32 equipos de clubes de primer nivel seleccionados en función de su desempeño en la liga nacional, lo que aumenta la intriga.  

Este evento electrizante trasciende los deportes y se convierte en una celebración de la unidad, la cultura y el orgullo nacional. Los fanáticos, vestidos con los colores de sus países, crean una atmósfera eléctrica, lo que hace que el torneo sea tanto sobre los espectadores como sobre los jugadores. Financieramente, la Liga de Campeones es un salvavidas para los clubes, ya que aumenta los ingresos y ofrece oportunidades transformadoras. Sin embargo, genera debates sobre las disparidades de riqueza en el fútbol europeo.  

La Liga de Campeones es sinónimo de rivalidades históricas, triunfos de los desvalidos y brillantez individual. Para los jugadores, representa la cima de su carrera, mientras que para los fanáticos, es un fenómeno cultural. El himno y los rituales icónicos enriquecen la experiencia futbolística. En 200 palabras, la UEFA Champions League es el epítome de la excelencia del fútbol europeo, que ofrece momentos inolvidables, recompensas económicas y un impacto cultural único, con 32 clubes de primer nivel que se suman a su atractivo.

**Nombres de las tablas o ficheros a utilizar**:  
[uefa_2020.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/EQuW8d16tZJKpYZN6jR480sBgc1IYypAw9hGHKQARS560g?e=7TybBU)  
[uefa_2021.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ESG8YBrjD3tBky3aGT0MwIABNt_PQxFVVYVlWMnZYUmV8g?e=VrYHtX)  
[uefa_2022.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/Ecf8O8U8sxJLoW9Bkd2ZVzgBb_gZc35mNcprnR9FMlzliQ?e=Vn3a6v)  
- Todas las tablas tienen las mismas columnas y data types

| Column | Definition | Data type |
|--------|------------|-----------|
| `STAGE`| Stage of the March | `VARCHAR(50)` |
| `DATE` | When the match occurred. | `DATE` |
| `PENS` | Did the match end with penalty | `VARCHAR(50)` |
| `PENS_HOME_SCORE` | In case of penalty, score by home team | `VARCHAR(50)` |
| `PENS_AWAY_SCORE` | In case of penalty, score by away team | `VARCHAR(50)` |
| `TEAM_NAME_HOME` | Team home name | `VARCHAR(50)` |
| `TEAM_NAME_AWAY`| Team away  name | `VARCHAR(50)` |
| `TEAM_HOME_SCORE` | Team home score | `NUMBER` |
| `TEAM_AWAY_SCORE` | Team away score | `NUMBER` |
| `POSSESSION_HOME` | Ball possession for the home team | `FLOAT` |
| `POSSESSION_AWAY` | Ball possession for the away team | `FLOAT` |
| `TOTAL_SHOTS_HOME` | Number of shots by the home team | `NUMBER` |
| `TOTAL_SHOTS_AWAY` | Number of shots by the away team | `NUMBER`
| `SHOTS_ON_TARGET_HOME` | Total shot for home team | `FLOAT` |
| `SHOTS_ON_TARGET_AWAY` | Total shot for away team | `FLOAT` |
| `DUELS_WON_HOME` | duel win possession of ball - for home team | `NUMBER` |
| `DUELS_WON_AWAY` | duel win possession of ball - for away team | `NUMBER` 
| `PREDICTION_TEAM_HOME_WIN` | Probability of home team to win | `FLOAT` |
| `PREDICTION_DRAW` | Probability of draw | `FLOAT` |
| `PREDICTION_TEAM_AWAY_WIN` | Probability of away team to win | `FLOAT` |
| `LOCATION` | Stadium where the match was held | `VARCHAR(50)` |

### Requerimientos:  
-Puedes utilizar SQL o PySpark

In [0]:
# Cargar los archivos CSV en DataFrames
df_2020 = spark.read.csv("/FileStore/examenDataBricks/Caso_2/uefa_2020.csv", header=True, inferSchema=True)
df_2021 = spark.read.csv("/FileStore/examenDataBricks/Caso_2/uefa_2021.csv", header=True, inferSchema=True)
df_2022 = spark.read.csv("/FileStore/examenDataBricks/Caso_2/uefa_2022.csv", header=True, inferSchema=True)

# Unir los DataFrames
df_all = df_2020.union(df_2021).union(df_2022)
df_all.write.csv("/FileStore/examenDataBricks/Caso_2/df_all_combined.csv", header=True)

# Verificar la estructura del DataFrame combinado
df_all.printSchema()
df_all.show(5)


root
 |-- STAGE: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- PENS: boolean (nullable = true)
 |-- PENS_HOME_SCORE: boolean (nullable = true)
 |-- PENS_AWAY_SCORE: boolean (nullable = true)
 |-- TEAM_NAME_HOME: string (nullable = true)
 |-- TEAM_NAME_AWAY: string (nullable = true)
 |-- TEAM_HOME_SCORE: integer (nullable = true)
 |-- TEAM_AWAY_SCORE: integer (nullable = true)
 |-- POSSESSION_HOME: string (nullable = true)
 |-- POSSESSION_AWAY: string (nullable = true)
 |-- TOTAL_SHOTS_HOME: integer (nullable = true)
 |-- TOTAL_SHOTS_AWAY: integer (nullable = true)
 |-- SHOTS_ON_TARGET_HOME: integer (nullable = true)
 |-- SHOTS_ON_TARGET_AWAY: integer (nullable = true)
 |-- DUELS_WON_HOME: string (nullable = true)
 |-- DUELS_WON_AWAY: string (nullable = true)
 |-- PREDICTION_TEAM_HOME_WIN: string (nullable = true)
 |-- PREDICTION_DRAW: string (nullable = true)
 |-- PREDICTION_TEAM_AWAY_WIN: string (nullable = true)
 |-- LOCATION: string (nullable = true)

+----------

1. Encuentra los 3 equipos que anotaron más goles jugando en su estadio en la UEFA Champions League 2020-21. El resultado debe contener dos columnas: TEAM_NAME_HOME y TEAM_HOME_SCORE ordenadas en orden descendente de TEAM_HOME_SCORE. Guarda la consulta (o el notebook) como TEAM_HOME_WITH_MOST_GOALS.

In [0]:
df_filtered_scores = df_all.filter(df_all.TEAM_HOME_SCORE.isNotNull())
# Seleccionar únicamente la columna TEAM_HOME_SCORE
df_filtered_scores.select("TEAM_HOME_SCORE").display()

TEAM_HOME_SCORE
0
2
2
1
1
0
1
0
0
0


In [0]:
# Filtrar los partidos donde el equipo juega en su estadio (como local)
df_filtered = df_all.filter(df_all.TEAM_HOME_SCORE.isNotNull())

# Convertir los goles a enteros, manejando los valores no numéricos
df_filtered = df_filtered.withColumn(
    "TEAM_HOME_SCORE", 
    F.when(F.col("TEAM_HOME_SCORE").rlike("^\d+$"), F.col("TEAM_HOME_SCORE").cast(IntegerType()))
    .otherwise(0)  # Reemplazar valores no numéricos por 0
)

# Agrupar por el equipo local y sumar los goles anotados en su estadio
df_goals_home = df_filtered.groupBy("TEAM_NAME_HOME").agg(
    F.sum("TEAM_HOME_SCORE").alias("TOTAL_GOALS") 
)

# Ordenar por los goles anotados en orden descendente
df_goals_home_sorted = df_goals_home.orderBy("TOTAL_GOALS", ascending=False)

# Obtener los 3 equipos con más goles
df_top_3_home_goals = df_goals_home_sorted.limit(3)

# Mostrar el resultado
df_top_3_home_goals.show()

# Guardar el resultado en un nuevo archivo
df_top_3_home_goals.write.csv("/FileStore/examenDataBricks/Caso_2/filtered_results/csv_1.csv", header=True)


+---------------+-----------+
| TEAM_NAME_HOME|TOTAL_GOALS|
+---------------+-----------+
|    Real Madrid|         26|
|Manchester City|         25|
|        Chelsea|         20|
+---------------+-----------+



2. Encuentra el equipo con posesión mayoritaria la mayor cantidad de veces durante la UEFA Champions League 2021-22. El resultado debe incluir dos columnas: TEAM_NAME y GAME_COUNT, que es la cantidad de veces que el equipo tuvo posesión mayoritaria durante un partido de fútbol. Guarda esta consulta (o el notebook) como TEAM_WITH_MAJORITY_POSSESSION

In [0]:
# Filtrar los partidos de la UEFA Champions League 2021-22 (suponiendo que ya cargaste los datos de esta temporada)
df_2021_22 = df_2021.union(df_2022)
# Crear una nueva columna para determinar qué equipo tuvo mayor posesión
df_possession_majority = df_2021_22.withColumn(
    "TEAM_WITH_MAJORITY_POSSESSION",
    F.when(df_2021_22["POSSESSION_HOME"] > df_2021_22["POSSESSION_AWAY"], df_2021_22["TEAM_NAME_HOME"])
    .otherwise(df_2021_22["TEAM_NAME_AWAY"])
)

# Contar la cantidad de veces que un equipo tuvo posesión mayoritaria
df_team_majority_possession_count = df_possession_majority.groupBy("TEAM_WITH_MAJORITY_POSSESSION").agg(
    F.count("TEAM_WITH_MAJORITY_POSSESSION").alias("GAME_COUNT")
)

# Renombrar las columnas para que sean TEAM_NAME y GAME_COUNT
df_team_majority_possession_count = df_team_majority_possession_count.withColumnRenamed(
    "TEAM_WITH_MAJORITY_POSSESSION", "TEAM_NAME"
)

# Ordenar por el número de veces que un equipo tuvo posesión mayoritaria
df_team_majority_possession_count_sorted = df_team_majority_possession_count.orderBy("GAME_COUNT", ascending=False)

# Mostrar el resultado
df_team_majority_possession_count_sorted.show()

# Guardar el resultado en un nuevo archivo
df_team_majority_possession_count_sorted.write.csv("/FileStore/examenDataBricks/Caso_2/filtered_results/csv_2.csv", header=True)


+-------------------+----------+
|          TEAM_NAME|GAME_COUNT|
+-------------------+----------+
|          Liverpool|        15|
|        Real Madrid|        13|
|            Chelsea|        13|
|    Manchester City|        11|
|               Ajax|         9|
|            Benfica|         9|
|          Barcelona|         8|
|           Man City|         7|
|             Bayern|         7|
|           Juventus|         7|
|Paris Saint-Germain|         7|
|             Napoli|         7|
|      Bayern Munich|         7|
|           Dortmund|         6|
|          Marseille|         6|
|         Man United|         5|
|              Paris|         5|
|              Milan|         5|
|           FC Porto|         5|
|            Sevilla|         5|
+-------------------+----------+
only showing top 20 rows



3. Encuentra la lista de equipos de cada fase del juego que ganaron el duelo en un partido pero terminaron perdiendo el juego en el Campeonato de la UEFA 2022-23. El resultado debe contener dos columnas: STAGE y TEAM_LOST. Guarda la consulta (o el notebook) como TEAM_WON_DUEL_LOST_GAME_STAGE_WISE.

In [0]:
from pyspark.sql import functions as F

# Filtrar los datos de la UEFA Champions League 2022-23
df_2022_filtered = df_2022

# Filtrar los partidos en los que un equipo ganó el duelo pero perdió el juego
df_duel_won_lost_game = df_2022_filtered.withColumn(
    "TEAM_WON_DUEL",
    F.when(
        # el equipo local gana mas duelos, pero pierde 
        (df_2022_filtered["DUELS_WON_HOME"] > df_2022_filtered["DUELS_WON_AWAY"]) & 
        (df_2022_filtered["TEAM_HOME_SCORE"] < df_2022_filtered["TEAM_AWAY_SCORE"]), df_2022_filtered["TEAM_NAME_HOME"]
    ).when(
        # el equipo visitante gana mas duelos, pero pierde
        (df_2022_filtered["DUELS_WON_AWAY"] > df_2022_filtered["DUELS_WON_HOME"]) & 
        (df_2022_filtered["TEAM_AWAY_SCORE"] < df_2022_filtered["TEAM_HOME_SCORE"]), df_2022_filtered["TEAM_NAME_AWAY"]
    ).otherwise(None)
)

# Filtrar solo los casos donde un equipo ganó el duelo pero perdió el juego
df_duel_won_lost_game_filtered = df_duel_won_lost_game.filter(df_duel_won_lost_game["TEAM_WON_DUEL"].isNotNull())

# Seleccionar las columnas STAGE y el equipo que ganó el duelo pero perdió el juego
df_result = df_duel_won_lost_game_filtered.select("STAGE", "TEAM_WON_DUEL").withColumnRenamed(
    "TEAM_WON_DUEL", "TEAM_LOST"
)

# Mostrar el resultado
df_result.show()

# Guardar el resultado en un nuevo archivo
df_result.write.csv("/FileStore/examenDataBricks/Caso_2/filtered_results/team_won_duel_lost_game_stage_wise.csv", header=True)


+--------------------+-----------------+
|               STAGE|        TEAM_LOST|
+--------------------+-----------------+
| Group stage: Mat...|          Chelsea|
| Group stage: Mat...|        K�benhavn|
| Group stage: Mat...|         Juventus|
| Group stage: Mat...|    Maccabi Haifa|
| Group stage: Mat...|          Rangers|
| Group stage: Mat...|        Liverpool|
| Group stage: Mat...|            Porto|
| Group stage: Mat...| Bayer Leverkusen|
| Group stage: Mat...|   Viktoria Plze?|
| Group stage: Mat...|        Marseille|
| Group stage: Mat...|             Ajax|
| Group stage: Mat...|  Atletico Madrid|
| Group stage: Mat...|        Barcelona|
| Group stage: Mat...|Tottenham Hotspur|
| Group stage: Mat...|       RB Leipzig|
| Group stage: Mat...|          Rangers|
| Group stage: Mat...|  Atletico Madrid|
| Group stage: Mat...| Bayer Leverkusen|
| Group stage: Mat...|   Viktoria Plze?|
| Group stage: Mat...|        Barcelona|
+--------------------+-----------------+
only showing top